In [1]:
import requests
def download_data()->dict:
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    response = requests.get(url)
    if response.status_code == 200:
        print("下載成功")
    return response.json()


In [2]:
def jsonDict_csvList(json)->list[dict]:
    '''
    - 傳入josn的資料結構
    - 取出需要的資料
    - 組合成list[dict]
    '''
    location = json['cwbopendata']['dataset']['location']
    weather_list = []
    for item in location:
        city_item = {}
        city_item['城市'] = item['locationName']
        city_item['啟始時間'] = item['weatherElement'][1]['time'][0]['startTime']
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        weather_list.append(city_item)
    return weather_list

In [3]:
json_data = download_data()
csv_list = jsonDict_csvList(json_data)
csv_list

下載成功


[{'城市': '臺北市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 34.0,
  '最低溫度': 30.0,
  '感覺': '悶熱至易中暑'},
 {'城市': '新北市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 33.0,
  '最低溫度': 30.0,
  '感覺': '悶熱'},
 {'城市': '桃園市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 33.0,
  '最低溫度': 30.0,
  '感覺': '悶熱'},
 {'城市': '臺中市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 33.0,
  '最低溫度': 31.0,
  '感覺': '悶熱'},
 {'城市': '臺南市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 32.0,
  '最低溫度': 30.0,
  '感覺': '悶熱'},
 {'城市': '高雄市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 33.0,
  '最低溫度': 30.0,
  '感覺': '悶熱至易中暑'},
 {'城市': '基隆市',
  '啟始時間': '2023-08-12T12:00:00+08:00',
  '結束時間': '2023-08-12T18:00:00+08:00',
  '最高溫度': 32.0,
  '最低溫度': 30.0,
  '感覺': '悶熱'},
 {'城市': '新竹縣

In [4]:
import csv
def save_csv(data:list[dict],fileName) -> None:
    '''
    - 將list[dict]儲存
    - 參數fileName要儲存的檔案名
    '''
    with open(fileName,mode='w',encoding='utf-8',newline='') as file:
        fieldnames = ['城市', '啟始時間','結束時間','最高溫度','最低溫度','感覺']
        writer = csv.DictWriter(file,fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

In [7]:
json_data = download_data() #下載資料
csv_list = jsonDict_csvList(json_data) #解析json的資料成為csv的結構
save_csv(csv_list,"今天天氣.csv") #儲存資料

下載成功


In [13]:
from datetime import datetime
import pytz
def get_csvName()->str:
    '''
    -取得台灣目前year-month-day.csv
    '''
    taiwan_timezone = pytz.timezone("Asia/Taipei")

    current_date = datetime.now(taiwan_timezone)
    fileName = f"{current_date.year}-{current_date.month}-{current_date.day}.csv"
    return fileName                                                     

In [14]:
csvFileName = get_csvName()
csvFileName

'2023-8-12.csv'

In [15]:
import os
current_cwd = os.path.abspath(os.getcwd())
abs_file_path = os.path.join(current_cwd,'data',csvFileName)
abs_file_path

'/workspaces/jeng/taiwan weather/data/2023-8-12.csv'